In [43]:
import tensorflow as tf
import datetime
import numpy as np
import matplotlib.pyplot as plt
from tools.models import AE
from tools.loadData import DataSets
from tensorflow.keras import optimizers
import os

In [44]:
lr=1e-3
batch_size=128
classes=10
batch_size=256
him_dims=20
dataset_name='mnist'

**load fashionMnist Data**

In [45]:
datasets=DataSets()
train_db,val_db,test_db=datasets.load_data(dataset_name,batch_size,classes)
x,y=next(iter(train_db))
print(x.shape,y.shape)

(256, 28, 28) (256, 10)


**load model**

In [46]:
model=AE(128)
model.build(input_shape=(None,28*28))
model.summary()

current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
log_dir = r'E:/pythonProject/DeepLearning/resources/logs/auto_encoding/' + current_time
summary_writer = tf.summary.create_file_writer(log_dir)

Model: "ae_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential_14 (Sequential)   (None, 128)               566144    
_________________________________________________________________
sequential_15 (Sequential)   (None, 784)               566800    
Total params: 1,132,944
Trainable params: 1,132,944
Non-trainable params: 0
_________________________________________________________________


In [47]:
import PIL.Image as Image

def save_img(imgs,name):
    new_img=Image.new('L',(280,280))
    index=0
    for i in range(0,280,28):
        for j in range(0,280,28):
            im=imgs[index]
            im=Image.fromarray(im,mode='L')
            new_img.paste(im,(i,j))
            index+=1
    new_img.save(name)
    return new_img


**trainModel**


In [48]:
optimizer=optimizers.Adam(lr=lr)

total_step=0
for epoch in range(50):
    for step, (x,y) in enumerate(train_db):
        with tf.GradientTape() as tape:
            x=tf.reshape(x,[-1,784])
            logits=model(x)
            loss=tf.losses.binary_crossentropy(x,logits,from_logits=True)
            loss=tf.reduce_mean(loss)
            total_step+=1
            with summary_writer.as_default():
                tf.summary.scalar("train_loss",loss,total_step)

        grads=tape.gradient(loss,model.trainable_variables)
        optimizer.apply_gradients(zip(grads,model.trainable_variables))

        if step%100==0:
            print("step=%d,loss=%0.2f"%(step,float(loss)))

    # 每次epoch 显示图片
    # predict
    x,y=next(iter(test_db))
    logit=model(tf.reshape(x,[-1,784]))
    x_hat=tf.sigmoid(logit)

    x_hat=tf.reshape(x_hat,[-1,28,28])

    x_concat=tf.concat([x[:50],x_hat[:50]],axis=0)
    x_concat=x_concat.numpy()*255
    x_concat=x_concat.astype(np.uint8)

    # saveimage
    new_img=save_img(x_concat,os.path.join(r"E:\pythonProject\DeepLearning\resources\images\autoEncode_img",
                                           dataset_name,
                                           "epoch_%d.png"%epoch
                                           ))
    new_img=np.asarray(new_img)
    new_img=np.expand_dims(new_img,[0,3])
    with summary_writer.as_default():
        tf.summary.image("第%d个epoch的编码图"%(epoch+1),new_img,epoch)

step=0,loss=0.69
step=100,loss=0.16
step=0,loss=0.12
step=100,loss=0.10
step=0,loss=0.10
step=100,loss=0.09
step=0,loss=0.09
step=100,loss=0.09
step=0,loss=0.09
step=100,loss=0.08
step=0,loss=0.08
step=100,loss=0.08
step=0,loss=0.08
step=100,loss=0.08
step=0,loss=0.08
step=100,loss=0.08
step=0,loss=0.07
step=100,loss=0.08


KeyboardInterrupt: 